# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [1]:
from unityagents import UnityEnvironment
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

# select this option to load version 1(with a single agent) of the environment
fname = r"C:\Users\anu\ProjectData\Reacher_Windows_x86_64\Reacher.exe"
fname = r"C:\Users\anu\deep-reinforcement-learning\p2_continuous-control\Reacher_Windows_x86_64\Reacher.exe"
env = UnityEnvironment(file_name=fname)

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [3]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [4]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.23999999463558197


### 4. Train the agent with DDPG

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.


In [ ]:
import gym
import random
import torch
import numpy as np
import time
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

agent = Agent(state_size=state_size, action_size=action_size, random_seed=0)

n_episodes = 2000
scores_deque = deque(maxlen=100)
total_scores = []

print_every = 10
max_t = 1000
num_learn = 10
learn_every = 20

for i_episode in range(1, n_episodes+1):
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations
    scores = np.zeros(num_agents)
    agent.reset()
    start_time = time.time()
    t = 0
    while True:
        actions = agent.act(states)
        env_info = env.step(actions)[brain_name]        
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        
        for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
            agent.step(state, action, reward, next_state, done)
        
        states = next_states
        scores += env_info.rewards
        t += 1
        if t%learn_every == 0:
            for _ in range(num_learn):
                agent.start_learn()        
        
        if np.any(dones):
            break
    
    mean_score = np.mean(scores)
    max_score = np.max(scores)
    min_score = np.min(scores)
    scores_deque.append(mean_score)
    total_scores.append(mean_score)
    avg_score = np.mean(scores_deque)
    total_time = time.time() - start_time
    
    print('\rEpisode {}\tAverage Score: {:.2f}\tMin Score: {:.2f}\tMax Score: {:.2f}\tTime: {:.2f}'.format(i_episode, np.mean(scores_deque), min_score, max_score, total_time), end="")
    if avg_score >=30.0:
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        break
    if i_episode % print_every == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

C:\Users\anu\Anaconda3\envs\drlnd\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 1	Average Score: 0.58	Min Score: 0.58	Max Score: 0.58	Time: 4.14

C:\Users\anu\deep-reinforcement-learning\p2_continuous-control\ddpg_agent.py:104: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1)


Episode 10	Average Score: 0.53	Min Score: 0.10	Max Score: 0.10	Time: 24.25
Episode 20	Average Score: 0.79	Min Score: 0.31	Max Score: 0.31	Time: 24.20
Episode 30	Average Score: 1.33	Min Score: 1.48	Max Score: 1.48	Time: 24.13
Episode 40	Average Score: 1.66	Min Score: 1.04	Max Score: 1.04	Time: 24.81
Episode 50	Average Score: 2.06	Min Score: 3.99	Max Score: 3.99	Time: 25.33
Episode 60	Average Score: 2.15	Min Score: 1.66	Max Score: 1.66	Time: 24.40
Episode 70	Average Score: 2.25	Min Score: 1.93	Max Score: 1.93	Time: 25.51
Episode 80	Average Score: 2.42	Min Score: 3.66	Max Score: 3.66	Time: 26.54
Episode 90	Average Score: 2.56	Min Score: 8.35	Max Score: 8.35	Time: 25.22
Episode 100	Average Score: 2.80	Min Score: 6.77	Max Score: 6.77	Time: 28.41
Episode 110	Average Score: 3.12	Min Score: 4.35	Max Score: 4.35	Time: 26.86
Episode 120	Average Score: 3.37	Min Score: 3.26	Max Score: 3.26	Time: 26.39
Episode 130	Average Score: 3.54	Min Score: 3.40	Max Score: 3.40	Time: 25.71
Episode 140	Average S

Episode 1060	Average Score: 9.35	Min Score: 20.70	Max Score: 20.70	Time: 48.58
Episode 1070	Average Score: 9.63	Min Score: 24.67	Max Score: 24.67	Time: 48.40
Episode 1080	Average Score: 10.41	Min Score: 19.56	Max Score: 19.56	Time: 48.30
Episode 1090	Average Score: 11.09	Min Score: 23.77	Max Score: 23.77	Time: 49.25
Episode 1100	Average Score: 11.42	Min Score: 7.32	Max Score: 7.32	Time: 49.5338
Episode 1110	Average Score: 11.71	Min Score: 16.93	Max Score: 16.93	Time: 49.20
Episode 1120	Average Score: 12.87	Min Score: 20.72	Max Score: 20.72	Time: 48.27
Episode 1130	Average Score: 13.62	Min Score: 18.26	Max Score: 18.26	Time: 48.47
Episode 1140	Average Score: 12.83	Min Score: 17.19	Max Score: 17.19	Time: 48.58
Episode 1150	Average Score: 12.87	Min Score: 1.95	Max Score: 1.95	Time: 48.6450
Episode 1159	Average Score: 12.96	Min Score: 16.48	Max Score: 16.48	Time: 49.09

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 